In [191]:
import os
os.chdir('C:/Users/AGB/Desktop/WeCloud_Materials/Project/Subreddit_Comments')
print(os.getcwd())

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
import re

C:\Users\AGB\Desktop\WeCloud_Materials\Project\Subreddit_Comments


In [192]:
# Select applicable subreddits which have been preprocessed
subreddits = ['Anarchism','Conservative','LateStageCapitalism','Libertarian']

df = pd.DataFrame([])

for i in subreddits: 
    tempdf = pd.read_csv(i+'_processed_comments.csv')
    tempdf['subreddit'] = i
    df = pd.concat([df,tempdf])

#df = df.sample(frac=0.3, random_state=10)
df.reset_index(drop=True,inplace=True)
df['subreddit'].value_counts()/len(df)

Libertarian            0.472729
LateStageCapitalism    0.237279
Conservative           0.228904
Anarchism              0.061087
Name: subreddit, dtype: float64

In [193]:
from ast import literal_eval

# Map main subreddits to political orientation to train word2vec model
political_map = {'Libertarian':'Conservative','Conservative':'Conservative',
                'LateStageCapitalism':'Liberal','Anarchism':'Liberal'}

df['subreddit'] = df['subreddit'].map(political_map) 

tokens = [] 

for i in range(len(df['tokens'])):
    tokens.append(literal_eval(df['tokens'][i]))

df['tokens'] = tokens

In [194]:
from gensim.models import Word2Vec
# Define training data
conservative_sentences = df['tokens'][df['subreddit']=='Conservative']
liberal_sentences = df['tokens'][df['subreddit']=='Liberal']

# Train model
conservative_model = Word2Vec(conservative_sentences, size=50, window=5, min_count=2, workers=4, sg=0)
print('Conservative Model Trained on {:.4}k comments: {}'.format(len(conservative_sentences)/1e3,conservative_model))

liberal_model = Word2Vec(liberal_sentences, size=50, window=5, min_count=2, workers=4, sg=0)
print('Liberal Model Trained on {:.4}k comments: {}'.format(len(liberal_sentences)/1e3,liberal_model))

# Save model
conservative_model.save('conservative_model')
liberal_model.save('liberal_model')

Conservative Model Trained on 127.3k comments: Word2Vec(vocab=37295, size=50, alpha=0.025)
Liberal Model Trained on 54.15k comments: Word2Vec(vocab=26745, size=50, alpha=0.025)


In [195]:
# Load trained model
conservative_model = Word2Vec.load('conservative_model')
liberal_model = Word2Vec.load('liberal_model')

# Determine most similar words
similarity_terms = ['donald','trump','clinton','obama','russia','FBI','alt-right','republican','rnc','democrat','dnc','left','right','guns',
         'media','news','fox','CNN','new york times','huffington','ABC','CBS',
        'antifa','anarchy','libertarian','communist','jews','gay','trans','black','white','asian','southern']
n_terms = 10

# Find most similar terms for term in specified list above
conservative_terms = []
liberal_terms = []

top_n_cols = ['Top_'+str(x+1) for x in range(n_terms)]

for term in similarity_terms:
    try:
        conservative_terms.append(np.transpose(conservative_model.wv.most_similar(term, topn=n_terms))[0])
        liberal_terms.append(np.transpose(liberal_model.wv.most_similar(term, topn=n_terms))[0])
    except:
        conservative_terms.append(np.full(n_terms, np.nan))
        liberal_terms.append(np.full(n_terms, np.nan))
    
conservative_terms = pd.DataFrame(conservative_terms,columns= top_n_cols)
conservative_terms.insert(0,'terms',similarity_terms)
conservative_terms.insert(0,'subreddit','Conservative')

liberal_terms = pd.DataFrame(liberal_terms,columns= top_n_cols)
liberal_terms.insert(0,'terms',similarity_terms)
liberal_terms.insert(0,'subreddit','Liberal')

word_embedding_df = pd.concat([conservative_terms,liberal_terms])
word_embedding_df.reset_index(drop=True,inplace=True)

In [196]:
key_term = 'hitler'

# Examine similar terms by trained Word2Vec models on both conservative and liberal corpora
if key_term in similarity_terms:
    display(word_embedding_df[word_embedding_df['terms']==key_term])
else:
    print("Top Conservative Terms Similar to ",key_term,":",np.transpose(conservative_model.wv.most_similar(key_term, topn=n_terms))[0])
    print()
    print("Top Liberal Terms Similar to ",key_term,":",np.transpose(liberal_model.wv.most_similar(key_term, topn=n_terms))[0])

Top Conservative Terms Similar to  hitler : ['mao' 'stalin' 'antifa' 'nazi' 'charlottesville' 'fascist' 'spencer'
 'adolf' 'fascists' 'nationalist']

Top Liberal Terms Similar to  hitler : ['christian' 'thatcher' 'popular' 'reagan' 'atheist' 'presidents'
 'holocaust' 'paul' 'jew' 'freaked']


In [197]:
# Word vector addition/subtraction with key terms
positive = ['trump','anarchist']
negative = []

print(conservative_model.wv.most_similar(positive = positive, negative = negative, topn=1))
print(liberal_model.wv.most_similar(positive = positive, negative = negative, topn=1))

[('-lolbertarian', 0.7825086712837219)]
[('leftist', 0.944356381893158)]
